# Advanced Geospatial Plotting

Import Libraries

In [1]:
import pandas as pd
import os
from keplergl import KeplerGl
from pyproj import CRS
import numpy as np
from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv('citibike_weather_cleaned_2022.csv', index_col = 0)

C:\Users\beaac\AppData\Local\Temp\ipykernel_30032\3026167464.py:1: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('citibike_weather_cleaned_2022.csv', index_col = 0)


In [3]:
df. columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'start_time', 'date', 'avgTemp', '_merge'],
      dtype='object')

In [4]:
df.dtypes

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
start_time             object
date                   object
avgTemp               float64
_merge                 object
dtype: object

In [5]:
df['ride_id'] = df['ride_id'].astype('category')
df['start_station_id'] = df['start_station_id'].astype('category')
df['end_station_id'] = df['end_station_id'].astype('category')
df['start_station_name'] = df['start_station_name'].astype('category')
df['end_station_name'] = df['end_station_name'].astype('category')
df['member_casual'] = df['member_casual'].astype('category')
df['_merge'] = df['_merge'].astype('category')

## Data Preprocessing

In [6]:
# Create new column with the value 1.
# create a new aggregated dataframe that contains 3 coumns: starting station, ending station and count trips

df['value'] = 1
df_group = df.groupby(['start_station_name', 'end_station_name'])['value'].count().reset_index()


C:\Users\beaac\AppData\Local\Temp\ipykernel_30032\1025877748.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_group = df.groupby(['start_station_name', 'end_station_name'])['value'].count().reset_index()


In [7]:
df_group

,start_station_name,end_station_name,value
0,1 Ave & E 110 St,1 Ave & E 110 St,791
1,1 Ave & E 110 St,1 Ave & E 16 St,0
2,1 Ave & E 110 St,1 Ave & E 18 St,2
3,1 Ave & E 110 St,1 Ave & E 30 St,4
4,1 Ave & E 110 St,1 Ave & E 39 St,1
...,...,...,...
3241996,Yankee Ferry Terminal,Wyckoff St & 3 Ave,0
3241997,Yankee Ferry Terminal,Wyckoff St & Nevins St,0
3241998,Yankee Ferry Terminal,Wythe Ave & Metropolitan Ave,0
3241999,Yankee Ferry Terminal,Yankee Ferry Terminal,5759


In [8]:
print(df_group['value'].sum())
print(df.shape)

29768282
(29768282, 18)


In [9]:
df_group['value'].describe()

count    3.242001e+06
mean     9.182071e+00
std      5.713709e+01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      1.000000e+00
max      1.204100e+04
Name: value, dtype: float64

In [10]:
#Merge coordinates directly rom original df
start_coords = df[['start_station_name', 'start_lat', 'start_lng']].drop_duplicates()
end_coords = df[['end_station_name', 'end_lat', 'end_lng']].drop_duplicates()

In [11]:
start_coords_check = df[['start_station_name', 'start_lat', 'start_lng']]
print(start_coords_check['start_station_name'].value_counts().head(10))


start_station_name
W 21 St & 6 Ave            128823
West St & Chambers St      123045
Broadway & W 58 St         114040
6 Ave & W 33 St            106236
1 Ave & E 68 St            104685
Broadway & E 14 St          98656
Broadway & W 25 St          98237
University Pl & E 14 St     96941
Broadway & E 21 St          95533
W 31 St & 7 Ave             94035
Name: count, dtype: int64


In [12]:
start_coords_clean = df.groupby('start_station_name')[['start_lat', 'start_lng']].first().reset_index()
end_coords_clean = df.groupby('end_station_name')[['end_lat', 'end_lng']].first().reset_index()


C:\Users\beaac\AppData\Local\Temp\ipykernel_30032\2919225807.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  start_coords_clean = df.groupby('start_station_name')[['start_lat', 'start_lng']].first().reset_index()
C:\Users\beaac\AppData\Local\Temp\ipykernel_30032\2919225807.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  end_coords_clean = df.groupby('end_station_name')[['end_lat', 'end_lng']].first().reset_index()


In [13]:
df_final = df_group.merge(start_coords_clean, on='start_station_name', how='left')
df_final = df_final.merge(end_coords_clean, on='end_station_name', how='left')


## Kepler.gl map

In [19]:
top_starts = (
    df_final.groupby('start_station_name')['value']
    .sum()
    .sort_values(ascending=False)
    .head(100)
    .reset_index()
)


C:\Users\beaac\AppData\Local\Temp\ipykernel_30032\3703269001.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_final.groupby('start_station_name')['value']


In [20]:
df_top100 = df_final[df_final['start_station_name'].isin(top_starts['start_station_name'])]


In [21]:
# clean column names
df_top100.columns = df_top100.columns.str.lower().str.replace(" ", "_")


In [22]:
df_top100.to_csv("top_100_routes.csv", index=False)


In [23]:
df_final.to_csv('df_final_location_for_map.csv')

In [24]:
# Create KeplerGl instance

m = KeplerGl(height = 700, data={"data_1": df_top100})
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data_1':                    start_station_name              end_station_name  value  \
1841   …

## Map Customization Summary 
 to visualize the most popular bike routes in NYC i did new df with the top 100 routes

I choose for the first layer 
start:  fill color Fuchsia
End: fill color Orange
start and end arc: The same color of the start and end, for the soure: Fuchsia and for the Target: Orange.

to finalized i applied a filter
Column: Value
Threshold : Top 100 stations by trip volume

Filtering reduced clutter and emphasized meaningful patterns, especially around Midtown, Brooklyn Bridge Park and Central Park

